<a href="https://colab.research.google.com/github/NoCodeProgram/deepLearning/blob/main/transformer/selfAttention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/NoCodeProgram/deepLearning.git

Cloning into 'deepLearning'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 246 (delta 37), reused 0 (delta 0), pack-reused 139
Receiving objects: 100% (246/246), 12.32 MiB | 20.15 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [2]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Read the text file
with open('deepLearning/transformer/shakespeare.txt', 'r') as file:
    text = file.read()

# Tokenize the text  (this is very simple tokenizer, in reality you would use a more advanced one)
tokenizer = get_tokenizer('basic_english')
tokens = tokenizer(text)
unique_tokens = set(tokens)


In [3]:
stoi = { s:i for i,s in enumerate(unique_tokens)}
itos = { i:s for i,s in enumerate(unique_tokens)}
print(stoi)
print(itos)

vocab_size = len(unique_tokens)
print(vocab_size)

{'consum': 0, 'derive': 1, 'sinks': 2, 'victor': 3, 'receivest': 4, 'girded': 5, 'riches': 6, 'builded': 7, 'follows': 8, 'counting': 9, 'beauteous': 10, 'owes': 11, 'advised': 12, 'walks': 13, 'dark': 14, 'knowing': 15, 'worms': 16, 'goodness': 17, 'foiled': 18, 'yet': 19, 'strained': 20, 'pitying': 21, 'hawks': 22, 'face': 23, 'eyed': 24, 'gazed': 25, 'lest': 26, 'thrivers': 27, 'adding': 28, 'proclaims': 29, 'painting': 30, 'friends': 31, 'swart-complexioned': 32, 'balmy': 33, 'asked': 34, 'trouble': 35, 'all-oblivious': 36, 'interchange': 37, 'writes': 38, 'said': 39, 'lovely': 40, 'arise': 41, 'crawls': 42, 'forgotten': 43, 'dulling': 44, 'seeming': 45, 'thinking': 46, 'divide': 47, 'willingly': 48, 'unbred': 49, 'cry': 50, 'receipt': 51, 'cries': 52, 'becoming': 53, 'draw': 54, 'whoe': 55, 'torture': 56, 'touch': 57, 'praise': 58, 'death-bed': 59, 'wires': 60, 'tempting': 61, 'bones': 62, 'swift-footed': 63, 'have': 64, 'niggarding': 65, 'could': 66, 'short-numbered': 67, 'threes

In [4]:
sentence = "i love you all"
indices = [stoi[word] for word in sentence.split()]
print(indices)

import torch.nn as nn

embedding_dim = 20
embedding = nn.Embedding(vocab_size, embedding_dim)

embedded_sentence = embedding(torch.tensor(indices))
print(embedded_sentence)


[1971, 989, 3098, 1003]
tensor([[ 0.1311,  0.8066,  0.4535,  0.0437, -0.2358,  0.5019, -0.4935, -1.2413,
          0.9908,  0.7749, -1.9193, -0.2777,  0.5189,  1.8600, -0.1343, -0.8929,
         -0.7727,  0.2107, -0.3738, -0.0824],
        [-0.2528, -2.1077,  0.1738, -0.7542,  0.0117, -0.6834,  0.0045,  0.9915,
         -0.6750, -0.6922,  0.1935,  0.0421,  0.4490, -1.0703, -0.9123,  0.6872,
          0.8562, -0.0615,  1.1582,  0.5923],
        [ 0.6818, -0.5055,  0.8669, -0.2833, -0.3931,  0.0202,  1.3754,  0.0614,
          0.9829, -0.7720, -0.0871,  0.2907, -0.0752, -0.4942,  1.1150,  1.6068,
          0.9314,  0.2309,  0.7592,  0.4716],
        [-0.6517,  0.3482, -1.3880, -0.5168, -2.4068, -0.4803, -0.7880, -1.7925,
         -0.4887,  0.4381, -0.0233,  0.1578, -1.0524, -0.1170, -0.9956,  0.5423,
         -0.6315, -2.1629, -1.3073,  0.6996]], grad_fn=<EmbeddingBackward0>)


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, embed_dim, atten_dim):
        super().__init__()
        self.query = nn.Linear(embed_dim, atten_dim, bias=False)
        self.key = nn.Linear(embed_dim, atten_dim, bias=False)
        self.value = nn.Linear(embed_dim, atten_dim, bias=False)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        scores = torch.matmul(query, key.transpose(-2, -1))
        scores = scores / torch.sqrt(torch.tensor(key.size(-1), dtype=torch.float))

        attention_weights = F.softmax(scores, dim=-1)
        weighted_values = torch.matmul(attention_weights, value)

        return weighted_values

In [6]:
# Create an instance of SelfAttention
head_dim = 5
self_attention = SelfAttention(embedding_dim, head_dim)

output = self_attention(embedded_sentence)
print(output)

tensor([[-0.0116,  0.0917, -0.2184,  0.0859, -0.0982],
        [ 0.0938, -0.1448, -0.0445, -0.1214,  0.1495],
        [ 0.0348, -0.0226, -0.1094, -0.0004,  0.0756],
        [ 0.0424, -0.0722, -0.1149, -0.0388,  0.0268]], grad_fn=<MmBackward0>)
